In [1]:
import geopandas as gpd
import os
import pandas as pd
import pathlib
import warnings
from typing import List, Tuple
warnings.filterwarnings("ignore")

import os
import sys
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname('__file__'), '..')))
from src.location.location import Location, ETAB_NAMES

#from osgeo import gdal
from shapely.geometry import Point

In [2]:
dir_geo = pathlib.Path('../data/geolocalisation')
ETABS = [etab[1] for etab in ETAB_NAMES]
# ETABS = ['CHU Dijon']
# COM = ['Cepoy', 'Chatou']
# DEPARTEMENT = ['45', '78', '91']
DATE = '2019-01-01'
FILES = ["base-ic-activite-residents-2019.xlsx",
            "base-ic-couples-familles-menages-2019.xlsx",
            "base-ic-diplomes-formation-2019.xlsx",
            "base-ic-evol-struct-pop-2019.xlsx",
            "base-ic-logement-2019.xlsx"]

In [3]:
code_geo_influence = []
for etab in ETABS:
    loc = Location(etab)
    code_geo_influence += loc.get_list_code()
code_geo_influence = list(set(code_geo_influence))
    

TypeError: 'NoneType' object is not subscriptable

In [114]:
code_geo_influence

['21610',
 '21380',
 '21760',
 '21640',
 '21C01',
 '21160',
 '21270',
 '21800',
 '21310',
 '21110',
 '21910',
 '21700',
 '21300',
 '21600',
 '21121',
 '21850',
 '21130',
 '21120',
 '21320',
 '21260',
 '21170',
 '21000',
 '21220',
 '21410',
 '21490',
 '21470',
 '21240',
 '21560',
 '21540',
 '21580',
 '21370']

In [115]:
city_influence = []
df_city = pd.read_excel(dir_geo / 'codes postaux_PMSI_pop_ATIH_2023.xlsx', usecols='B:E')
#df_city = df_city.loc[df_city['INSEE'].str.contains('|'.join(code_geo_influence))]
for code in code_geo_influence:
    df_city_filtered = df_city.loc[df_city['Code géographique PMSI 2023'].str.contains(code)]
    #print(df_city_filtered)
    city_influence += df_city_filtered['Libellé poste'].tolist()
city_influence = list(set(city_influence))
len(city_influence)

32

In [116]:
city_influence

['MESSIGNY-ET-VANTOUX',
 'NOIRON-SOUS-GEVREY',
 'VAROIS-ET-CHAIGNOT',
 'SOMBERNON',
 'SELONGEY',
 'SAINT-MARTIN-DU-MONT',
 'PONTAILLER-SUR-SAÔNE',
 'FONTAINE-LÈS-DIJON',
 'CHENÔVE',
 'VERREY-SOUS-SALMAISE',
 'SAINT-APOLLINAIRE',
 'GRANCEY-LE-CHÂTEAU-NEUVELLE',
 'AUXONNE',
 'POUILLY-EN-AUXOIS',
 'LOSNE',
 'ARC-SUR-TILLE',
 'IS-SUR-TILLE',
 'MARSANNAY-LA-CÔTE',
 'LAMARCHE-SUR-SAÔNE',
 'PLOMBIÈRES-LÈS-DIJON',
 'BRAZEY-EN-PLAINE',
 'FONTAINE-FRANÇAISE',
 'LONGVIC',
 'GEVREY-CHAMBERTIN',
 'FLEUREY-SUR-OUCHE',
 'NUITS-SAINT-GEORGES',
 'DIJON',
 'GILLY-LÈS-CÎTEAUX',
 'GENLIS',
 'MIREBEAU-SUR-BÈZE',
 'TALANT',
 'CHEVIGNY-SAINT-SAUVEUR']

## Features au niveau des IRIS

In [117]:
# Fichier de contours IRIS : https://geoservices.ign.fr/contoursiris
geoiris = gpd.read_file(dir_geo / 'iris' / 'CONTOURS-IRIS.shp')

In [118]:
geoiris

,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS,geometry
0,72191,Mayet,0000,721910000,Mayet,Z,"POLYGON ((497887.400 6747662.400, 497907.200 6..."
1,77248,Lesches,0000,772480000,Lesches,Z,"POLYGON ((685757.700 6868592.300, 685832.500 6..."
2,51426,Péas,0000,514260000,Péas,Z,"POLYGON ((757292.900 6847598.400, 757334.300 6..."
3,81199,Padiès,0000,811990000,Padiès,Z,"POLYGON ((646146.900 6328153.200, 646124.900 6..."
4,59225,Feignies,0102,592250102,Sud,H,"POLYGON ((767596.000 7022269.100, 767242.200 7..."
...,...,...,...,...,...,...,...
48585,38545,Vif,0101,385450101,Centre Ville,H,"POLYGON ((910757.300 6442012.700, 910509.600 6..."
48586,74063,Châtel,0000,740630000,Châtel,Z,"POLYGON ((991445.300 6579384.800, 991285.500 6..."
48587,95101,Bray-et-Lû,0000,951010000,Bray-et-Lû,Z,"POLYGON ((601990.800 6893254.000, 602029.800 6..."
48588,38403,Saint-Jean-d'Hérans,0000,384030000,Saint-Jean-d'Hérans,Z,"POLYGON ((919902.300 6424004.200, 919912.200 6..."


In [119]:
# Coordinate Reference System : Lambert 93
geoiris.crs = "EPSG:2154"

In [120]:
geoiris['NOM_COM'] = geoiris['NOM_COM'].str.upper()
geoiris = geoiris.query('NOM_COM in @city_influence')

In [121]:
geoiris.sort_values(by='INSEE_COM', inplace=True)

In [122]:
geoiris

,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS,geometry
1950,01374,SAINT-MARTIN-DU-MONT,0000,013740000,Saint-Martin-du-Mont,Z,"POLYGON ((879337.900 6561798.500, 879677.900 6..."
2997,05130,SAINT-APOLLINAIRE,0000,051300000,Saint-Apollinaire,Z,"POLYGON ((967446.100 6388720.500, 967427.100 6..."
23439,21021,ARC-SUR-TILLE,0000,210210000,Arc-sur-Tille,Z,"POLYGON ((868837.900 6693437.900, 868833.400 6..."
17941,21038,AUXONNE,0101,210380101,Colombiere et Exterieurs,H,"POLYGON ((880536.500 6680144.700, 880656.500 6..."
12579,21038,AUXONNE,0103,210380103,Quartier Neuf et Extérieurs,H,"POLYGON ((880676.200 6679169.500, 880730.700 6..."
...,...,...,...,...,...,...,...
39399,21617,TALANT,0103,216170103,Mail-Canzio-Jouvet,H,"POLYGON ((850656.400 6695168.300, 850644.200 6..."
35819,21617,TALANT,0102,216170102,Maronniers-Neruda,H,"POLYGON ((851317.600 6694557.100, 851330.700 6..."
30047,21657,VAROIS-ET-CHAIGNOT,0000,216570000,Varois-et-Chaignot,Z,"POLYGON ((859278.000 6696641.600, 859215.700 6..."
41405,21670,VERREY-SOUS-SALMAISE,0000,216700000,Verrey-sous-Salmaise,Z,"POLYGON ((824459.500 6705803.600, 824647.200 6..."


In [280]:
# for fic in ["base-ic-activite-residents-2019.xlsx", 
#             "base-ic-couples-familles-menages-2019.xlsx", 
#             "base-ic-diplomes-formation-2019.xlsx",
#             "base-ic-evol-struct-pop-2019.xlsx",
#             "base-ic-logement-2019.xlsx"]:
#     df = pd.read_excel(dir_geo / 'insee' / fic, header = 4, nrows=10)
#     df.dropna(axis=1, inplace=True)
#     dico = dict(zip(list(df.head(1).values[0]), df.columns))
#     for k in dico:
#         if k.startswith(('P19', 'C19')):
#             print(f'"{k}", # {dico[k]}')
#             #if k=='C19_MEN':
#             #    print(fic)

In [123]:
def get_insee_data(geoiris_init, commune):
    #geoiris = geoiris_init.copy(deep=True)
    geoiris_final = pd.DataFrame()
    for file in FILES:
        df = pd.read_excel(dir_geo / 'insee' / file, header = 5)
        df.rename(columns={'IRIS':'CODE_IRIS'}, inplace=True)
        df.dropna(axis=1, inplace=True)
        geoiris = geoiris_init.merge(df, how = 'left') 
        geoiris = geoiris.loc[geoiris.NOM_COM == commune]
        geoiris = geoiris.filter(regex=f'^(P{DATE[2:5]}|C{DATE[2:5]})')
        geoiris = geoiris.sum(axis=0).to_frame().T
        geoiris['date'] = DATE
        geoiris['date'].astype('datetime64[ns]')
        #geoiris.set_index('date', inplace=True)
        geoiris.reset_index(drop=True, inplace=True)
        common_columns = geoiris_final.columns.intersection(geoiris.columns)
        geoiris = geoiris.drop(columns=common_columns)
        geoiris_final = geoiris_final.join(geoiris, how='outer')
    geoiris_final['COM'] = commune
    geoiris_final['INSEE_COM'] = geoiris_init['INSEE_COM']
    return geoiris_final

In [ ]:
insee_df = pd.DataFrame()
for commune in city_influence:
    insee_df = pd.concat([insee_df, get_insee_data(geoiris, commune)], ignore_index=True)

insee_df = insee_df[['date'] + [col for col in insee_df.columns if col != 'date']]
insee_df

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1044a5210>>
Traceback (most recent call last):
  File "/Users/julienaudoux/Desktop/WORK(lab)/forecasting_models/.venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
insee_df.to_feather('insee.feather')